In [ ]:
from google.cloud import bigquery
import pandas as pd
PROJECT = "ballosodeuk"
bq = bigquery.Client(project=PROJECT)

In [ ]:
query = """
    WITH events AS (
    SELECT 
        TIMESTAMP(Event_Datetime) AS Event_Timestamp,
        Event_Date, 
        Airbridge_Device_ID,
        
        Event_Category, 
        Event_Action, 
        Event_Label, 
        Semantic_Event_Properties
    FROM 
        `ballosodeuk.airbridge_bq.app` 
    WHERE 
        (Event_Category = 'Ad Impression (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡')
        OR (Event_Category = 'Ad Click (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡')
    ),

    sorted_events AS (
    SELECT
        Event_Date,
        Event_Timestamp,
        Event_Category,
        Airbridge_Device_ID,
        LEAD(Event_Category) OVER (PARTITION BY Airbridge_Device_ID ORDER BY Event_Timestamp) AS next_category,
        LEAD(Event_Timestamp) OVER (PARTITION BY Airbridge_Device_ID ORDER BY Event_Timestamp) AS next_timestamp
    FROM
        events
    ),

    result_1 AS (
    SELECT
        Event_Date,
        Event_Timestamp,
        Event_Category,
        Airbridge_Device_ID
    FROM
        sorted_events
    WHERE
        Event_Category = 'Ad Impression (App)' 
        AND next_category = 'Ad Click (App)'
    ),

    result_2 AS (
    SELECT 
        e.Event_Timestamp,
        e.Event_Date, 
        e.Airbridge_Device_ID, 
        e.Event_Category, 
        e.Event_Action, 
        e.Event_Label,
        p.name as Product_Name, 
        p.position as Product_Position, 
        p.price as Product_Price
    FROM 
        events e,
        UNNEST(JSON_EXTRACT_ARRAY(e.Semantic_Event_Properties, '$.products')) AS prod_json
    CROSS JOIN 
        UNNEST([JSON_EXTRACT_SCALAR(prod_json, '$.position') = '0']) AS is_position_zero
    JOIN 
        UNNEST([STRUCT(
        JSON_EXTRACT_SCALAR(prod_json, '$.name') AS name,
        JSON_EXTRACT_SCALAR(prod_json, '$.position') AS position,
        JSON_EXTRACT_SCALAR(prod_json, '$.price') AS price
        )]) AS p
    WHERE 
        e.Event_Category = 'Ad Impression (App)' AND is_position_zero
    )

    SELECT 
    r1.Event_Date,
    r1.Event_Timestamp,
    r1.Airbridge_Device_ID,
    r1.Event_Category,
    r2.Event_Label,
    r2.Event_Action,
    r2.Product_Name,
    r2.Product_Position,
    r2.Product_Price
    FROM 
    result_1 r1
    JOIN 
    result_2 r2 ON r1.Event_Timestamp = r2.Event_Timestamp AND r1.Airbridge_Device_ID = r2.Airbridge_Device_ID
    ORDER BY 
    r1.Airbridge_Device_ID, r1.Event_Timestamp ASC;

"""


In [ ]:
query = """
WITH events AS (
  SELECT 
    TIMESTAMP(Event_Datetime) AS Event_Timestamp,
    Event_Date,
    User_ID,
    Airbridge_Device_ID, 
    Event_Category, 
    Event_Action, 
    Event_Label, 
    Semantic_Event_Properties
  FROM 
    `ballosodeuk.airbridge_bq.app` 
  WHERE 
    (Event_Category = 'Ad Impression (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡')
    OR (Event_Category = 'Ad Click (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡')
),

-- sorted_events AS (
--   SELECT
--     Event_Date,
--     Event_Timestamp,
--     Event_Category,
--     Airbridge_Device_ID,
--     LEAD(Event_Category) OVER (PARTITION BY Airbridge_Device_ID ORDER BY Event_Timestamp) AS next_category,
--     LEAD(Event_Timestamp) OVER (PARTITION BY Airbridge_Device_ID ORDER BY Event_Timestamp) AS next_timestamp
--   FROM
--     events
-- ),

-- result_1 AS (
--   SELECT
--     Event_Date,
--     Event_Timestamp,
--     Event_Category,
--     Airbridge_Device_ID
--   FROM
--     sorted_events
--   WHERE
--     Event_Category = 'Ad Impression (App)' 
--     AND next_category = 'Ad Click (App)'
--     AND TIMESTAMP_DIFF(next_timestamp, Event_Timestamp, HOUR) <= 5
-- ),

result_2 AS (
  SELECT 
    e.Event_Timestamp,
    e.Event_Date, 
    e.Airbridge_Device_ID,
    e.User_ID,
    e.Event_Category, 
    e.Event_Action, 
    e.Event_Label,
    p.name as Product_Name, 
    p.position as Product_Position, 
    p.price as Product_Price
  FROM 
    events e,
    UNNEST(JSON_EXTRACT_ARRAY(e.Semantic_Event_Properties, '$.products')) AS prod_json
  CROSS JOIN 
    UNNEST([JSON_EXTRACT_SCALAR(prod_json, '$.position') = '0']) AS is_position_zero
  JOIN 
    UNNEST([STRUCT(
      JSON_EXTRACT_SCALAR(prod_json, '$.name') AS name,
      JSON_EXTRACT_SCALAR(prod_json, '$.position') AS position,
      JSON_EXTRACT_SCALAR(prod_json, '$.price') AS price
    )]) AS p
  WHERE 
    e.Event_Category = 'Ad Impression (App)' AND is_position_zero
)

select *
from events
where Event_Category = 'Ad Click (App)'

"""

In [ ]:
config = bigquery.QueryJobConfig()
config.dry_run=True
job = bq.query(query,location='asia-northeast3',job_config=config)
print("This query will process {} bytes"
      .format(job.total_bytes_processed))
print("This query will Cost {} "
      .format(job.total_bytes_billed))


In [ ]:
df = bq.query(query,location='asia-northeast3').to_dataframe()
df